# Experiment 3
- Load the saved model and replace the output layer of the model, as well as the first two convolutional layers (keep the weights of all other layers).
- Train and evaluate the model on the cats and dogs dataset.

## Steps
Load the saved model from a file.

Freeze all layers except the first two convolutional layers and the output layer.

Replace the first two convolutional layers and the output layer.

Compile the model with a suitable loss function and optimizer.

Train and evaluate the model on the Cats vs. Dogs dataset.

### Load the saved model

In [1]:
import tensorflow as tf
from tensorflow import data as tf_data
from tensorflow import keras
from keras import layers

# Load the pre-trained model
saved_model_path = "models/experiment1_model.keras"
model = keras.models.load_model(saved_model_path)

### Modify the model

In [2]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        units = 1
    else:
        units = num_classes

    x = layers.Dropout(0.25)(x)
    # Change output layer for multi-class classification (120 dog breeds)
    outputs = layers.Dense(units, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [3]:
# Create a fresh model with the same architecture
fresh_model = make_model(input_shape=(180, 180) + (3,), num_classes=1)
base_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
base_model.trainable = False 

# Reset weights for layers 5 to 22
for i in range(5, 22):
    base_model.layers[i].set_weights(fresh_model.layers[i].get_weights())
    base_model.layers[i].trainable = True  # Make sure the layers are trainable

In [4]:
# Add new output layer (Binary Classification)
new_output = layers.Dense(1, activation="sigmoid")(base_model.output)

# Create a new model
new_model = keras.Model(inputs=base_model.input, outputs=new_output)

In [5]:
# Ensure the new model is compiled with the goof layer unfrozen
for i, layer in enumerate(new_model.layers):
    print(f"{i}: {layer.name} — Trainable: {layer.trainable}")

0: input_layer_1 — Trainable: False
1: rescaling_1 — Trainable: False
2: conv2d_4 — Trainable: False
3: batch_normalization_8 — Trainable: False
4: activation_8 — Trainable: False
5: activation_9 — Trainable: True
6: separable_conv2d_7 — Trainable: True
7: batch_normalization_9 — Trainable: True
8: activation_10 — Trainable: True
9: separable_conv2d_8 — Trainable: True
10: batch_normalization_10 — Trainable: True
11: max_pooling2d_3 — Trainable: True
12: conv2d_5 — Trainable: True
13: add_3 — Trainable: True
14: activation_11 — Trainable: True
15: separable_conv2d_9 — Trainable: True
16: batch_normalization_11 — Trainable: True
17: activation_12 — Trainable: True
18: separable_conv2d_10 — Trainable: True
19: batch_normalization_12 — Trainable: True
20: max_pooling2d_4 — Trainable: True
21: conv2d_6 — Trainable: True
22: add_4 — Trainable: False
23: activation_13 — Trainable: False
24: separable_conv2d_11 — Trainable: False
25: batch_normalization_13 — Trainable: False
26: activation_14

### Compile the model

In [6]:
new_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Modification for experiment 1
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(name="acc")],
)

### Create the dataset

In [7]:
# Create the dataset
image_size = (180, 180)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x), y))

# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Using 4684 files for validation.


### Train and evaluate the model

In [ ]:
epochs = 50
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="models/experiment3_epoch_{epoch}.keras")
]


new_model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=callbacks
)

new_model.save("models/experiment3_model.keras")


Epoch 1/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1007s 6s/step - acc: 0.5870 - loss: 0.7095 - val_acc: 0.5352 - val_loss: 0.6903
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 998s 7s/step - acc: 0.6797 - loss: 0.5930 - val_acc: 0.5186 - val_loss: 0.6941
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 929s 6s/step - acc: 0.7174 - loss: 0.5537 - val_acc: 0.4964 - val_loss: 0.7212
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 989s 7s/step - acc: 0.7347 - loss: 0.5270 - val_acc: 0.5083 - val_loss: 0.7819
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 945s 6s/step - acc: 0.7501 - loss: 0.5070 - val_acc: 0.6800 - val_loss: 0.5758
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1153s 8s/step - acc: 0.7729 - loss: 0.4798 - val_acc: 0.7737 - val_loss: 0.4780
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1307s 9s/step - acc: 0.7733 - loss: 0.4740 - val_acc: 0.7882 - val_loss: 0.4580
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 1193s 8s/step - acc: 0.7918 - loss: 0.4422 - val_acc: 0.7959 - val_loss: 0.4357
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 861s